## Disease Network 실습 using Python
* 진단 score 테이블 생성 예제 (모든 진단과 환자 고려)

### Requirement
* Dignosis progression network와 같이 이전 진단과 이후 진단에 대한 $D_i$ x $D_j$ Matrix를 생성한다.
<br/>
* Matrix의 score(점수)는 카운트의 백분율 값으로 환산한다.
<br/>

* 식은 다음과 같이 정의한다.
    * $B\over A$ ; $A$: 이전 진단 $D_i$의 총 카운트, $B$: 이전 진단 $D_i$을 가지고 있는 이후 진단 $D_j$의 총 카운트

<br/>

* **단, 환자 개개인의 반복 측정으로 인해 카운트 계산 중 다음 환자로 넘어갈 경우를 고려해야 한다.**

Import the Pandas, Numpy

In [1]:
import pandas as pd
import pandasgui as gui
import numpy as np

Load data set (.sas)

In [2]:
pd_sas = pd.read_sas('nsc2_edu_m20.sas7bdat', format='sas7bdat')
pd_sas

,RN_INDI,RN_KEY,RN_INST,MDCARE_STRT_DT,FORM_CD,SICK_SYM1,SICK_SYM2,OPRTN_YN,MDCARE_DD_CNT,VSHSP_DD_CNT,TOT_PRSC_DD_CNT,MCARE_RSLT_TYPE,ED_RC_TOT_AMT,EDC_SBA,EDC_INSUR_BRDN_AMT,SPCF_SYM_TYPE
0,3050415.0,b'20060000000008',203739.0,b'20060113',b'03',b'C20',NaN,b'0',1.0,1.0,0.0,b'1',10410.0,0.0,10410.0,b'V027'
1,6771465.0,b'20060000000011',876467.0,b'20060105',b'03',b'S335',NaN,b'0',1.0,1.0,0.0,b'1',14060.0,1500.0,12560.0,NaN
2,9238266.0,b'20060000000013',396247.0,b'20060118',b'03',b'R104',NaN,b'0',1.0,1.0,0.0,b'1',17780.0,5330.0,12450.0,NaN
3,2518883.0,b'20060000000015',192322.0,b'20060107',b'03',b'I10',b'I70',b'0',6.0,6.0,33.0,b'1',55690.0,9000.0,46690.0,NaN
4,2251892.0,b'20060000000020',503221.0,b'20060102',b'02',b'J189',b'J459',b'0',21.0,12.0,0.0,b'1',2650240.0,602890.0,2047350.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137862,9817912.0,b'20120000075216',256194.0,b'20121224',b'03',b'J029',b'J303',b'0',1.0,1.0,3.0,b'1',9220.0,2700.0,6520.0,NaN
137863,9799437.0,b'20120000075217',135151.0,b'20121208',b'03',b'J060',b'J304',b'0',1.0,1.0,3.0,b'1',11230.0,3300.0,7930.0,NaN
137864,9804004.0,b'20120000075218',252557.0,b'20121214',b'03',b'J00',b'J209',b'0',1.0,1.0,3.0,b'1',9210.0,2700.0,6510.0,NaN
137865,9799437.0,b'20120000075219',135151.0,b'20121226',b'03',b'J0141',b'J303',b'0',1.0,1.0,3.0,b'1',9210.0,2700.0,6510.0,NaN


필요 변수만 사용
* 필요 변수: 환자 ID, 방문 일자, 주증상1
* 환자 ID, 병원 방문 일자에 따라 정렬
* 변수 타입 변환

In [3]:
df_patients = pd_sas[['RN_INDI', 'MDCARE_STRT_DT', 'SICK_SYM1']]\
                    .sort_values(by=['RN_INDI', 'MDCARE_STRT_DT'])\
                    .astype({'RN_INDI':'int','MDCARE_STRT_DT':'str', 'SICK_SYM1':'str'})

df_patients.info()

<class 'pandas.core.frame.DataFrame'>
Index: 137867 entries, 585 to 137855
Data columns (total 3 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   RN_INDI         137867 non-null  int32 
 1   MDCARE_STRT_DT  137867 non-null  object
 2   SICK_SYM1       137867 non-null  object
dtypes: int32(1), object(2)
memory usage: 3.7+ MB


disease 데이터 출력

In [4]:
df_patients

,RN_INDI,MDCARE_STRT_DT,SICK_SYM1
585,1001978,20060102,H650
2201,1001978,20060222,J039
4685,1001978,20060302,K52
3830,1001978,20060303,H660
5834,1001978,20060401,J459
...,...,...,...
127868,9825078,20120908,J209
127866,9825078,20120919,J209
131250,9825078,20121017,J209
134682,9825078,20121121,J209


"I"를 포함하는 진단만 필터링

In [5]:
# df_patients = df_patients[df_patients['SICK_SYM1'].str.contains("I")]

# df_patients

각 진단이 가지고 있는 카운트 출력

In [6]:
df_patients['SICK_SYM1'].value_counts(sort=False)

SICK_SYM1
H650     308
J039    2574
K52       39
H660     341
J459    1056
        ... 
P59        1
Q659       2
P071       1
P011       1
S309       1
Name: count, Length: 3447, dtype: int64

각 환자가 가지고 있는 고유한 진단 출력 및 진단 배열 생성
* 총 3,447개

In [7]:
diagnosis = df_patients['SICK_SYM1'].unique()
print(diagnosis)
print(diagnosis.shape)

['H650' 'J039' 'K52' ... 'P071' 'P011' 'S309']
(3447,)


Id와 진단만 추출

In [8]:
df_patients = df_patients[['RN_INDI', 'SICK_SYM1']]

데이터 분할 후 좌우로 결합

In [9]:
prev = df_patients[:len(df_patients)-1] # 이전 데이터
prev.reset_index(inplace=True)

next = df_patients[1:] # 다음 데이터
next.reset_index(inplace=True)

concat = pd.concat([prev[['RN_INDI', 'SICK_SYM1']], 
                    next[['RN_INDI', 'SICK_SYM1']]], axis=1)
concat

,RN_INDI,SICK_SYM1,RN_INDI,SICK_SYM1
0,1001978,H650,1001978,J039
1,1001978,J039,1001978,K52
2,1001978,K52,1001978,H660
3,1001978,H660,1001978,J459
4,1001978,J459,1001978,J06
...,...,...,...,...
137861,9825078,J209,9825078,J209
137862,9825078,J209,9825078,J209
137863,9825078,J209,9825078,J209
137864,9825078,J209,9825078,J209


컬럼명 변경 후 prev ID와 next ID가 다르면 해당 행 삭제
* 개인 환자 고려

In [10]:
concat.columns = ['RN_INDI_PREV', 'SICK_SYM1_PREV',
                  'RN_INDI_NEXT', 'SICK_SYM1_NEXT']
concat.drop(concat[(concat['RN_INDI_PREV']) != (concat['RN_INDI_NEXT'])].index, inplace=True)
concat

,RN_INDI_PREV,SICK_SYM1_PREV,RN_INDI_NEXT,SICK_SYM1_NEXT
0,1001978,H650,1001978,J039
1,1001978,J039,1001978,K52
2,1001978,K52,1001978,H660
3,1001978,H660,1001978,J459
4,1001978,J459,1001978,J06
...,...,...,...,...
137861,9825078,J209,9825078,J209
137862,9825078,J209,9825078,J209
137863,9825078,J209,9825078,J209
137864,9825078,J209,9825078,J209


Numpy 변환

In [11]:
np_patients_concat = concat.to_numpy()
np_patients_concat

array([[1001978, 'H650', 1001978, 'J039'],
       [1001978, 'J039', 1001978, 'K52'],
       [1001978, 'K52', 1001978, 'H660'],
       ...,
       [9825078, 'J209', 9825078, 'J209'],
       [9825078, 'J209', 9825078, 'J209'],
       [9825078, 'J209', 9825078, 'J209']], dtype=object)

계산 방법
* **score matrix에서 행과 열 전체를 순회하는 것이 아닌 진단 쌍에서 가지고 있는 요소만 계산하여 행과 열에 넣어주기**


* 필요 변수
    * 각 진단에 대한 **진단명:진단번호** 를 가지는 딕셔너리 -> 진단 번호로 score matrix의 행과 열의 인덱스에 매핑 가능
    * 진단 쌍 배열 -> [ [이전 진단, 이후 진단], ...]
    * 이전 진단의 개수 배열 -> [$D_1$의 개수, $D_2$의 개수, ..., $D_I$ 개수]
     

In [12]:
# 진단 배열에서 진단에 대한 진단번호 매기기 (score matrix를 위함)
dig_dict = {diagnosis[i] : i for i in range(len(diagnosis))} # ex) {I10(KEY):0(VALUE), I12(KEY):1(VALUE) ...} 
print("dig_dict: \n", dig_dict)

# 진단 쌍 배열 생성: 좌우로 결합된 환자 데이터 프레임의 이전 이후 진단쌍 배열 생성, 행에 맞춰 추가 됨
pairs_number = np.array([[dig_dict[D[1]], dig_dict[D[3]]] for D in np_patients_concat]) # D[1]=이전 진단번호 || D[3]=이후 진단번호
print("pairs: \n", pairs_number)

# 기존 환자 데이터 프레임에서 고유한 값을 가진 진단의 개수 배열 생성 
np_patients = df_patients.to_numpy() #  -> numpy 변환
diagnosis_number = np.array([dig_dict[diag] for diag in np_patients[:,1]]) # dig_dict 변수의 진단번호(VALUE) 삽입
countsOfD_i = np.unique(diagnosis_number, return_counts=True)[1] # 고유 진단번호의 개수(이전 진단이 가지고 있는 개수와 같음) 

print("countsOfD_i: \n", countsOfD_i)

dig_dict: 
 {'H650': 0, 'J039': 1, 'K52': 2, 'H660': 3, 'J459': 4, 'J06': 5, 'K529': 6, 'J069': 7, 'J209': 8, 'H66': 9, 'J019': 10, 'A090': 11, 'J450': 12, 'J060': 13, 'J320': 14, 'J304': 15, 'H101': 16, 'H400': 17, 'J030': 18, 'H6500': 19, 'K27': 20, 'R104': 21, 'S3350': 22, 'J029': 23, 'L309': 24, 'H113': 25, 'H521': 26, 'L209': 27, 'J311': 28, 'S6250': 29, 'S6260': 30, 'S602': 31, 'S900': 32, 'J00': 33, 'L24': 34, 'L259': 35, 'L239': 36, 'B351': 37, 'M1281': 38, 'K219': 39, 'L0280': 40, 'J40': 41, 'J0190': 42, 'E039': 43, 'M961': 44, 'S335': 45, 'J189': 46, 'K30': 47, 'G43': 48, 'G442': 49, 'L249': 50, 'B028': 51, 'H010': 52, 'M511': 53, 'E785': 54, 'R11': 55, 'D139': 56, 'J020': 57, 'K297': 58, 'J36': 59, 'J18': 60, 'K259': 61, 'F_': 62, 'H612': 63, 'S903': 64, 'S818': 65, 'R508': 66, 'H669': 67, 'L739': 68, 'J180': 69, 'H041': 70, 'J042': 71, 'J068': 72, 'S934': 73, 'H409': 74, 'J45': 75, 'N300': 76, 'H408': 77, 'M501': 78, 'R32': 79, 'K599': 80, 'R42': 81, 'E02': 82, 'H40102': 83

Score Matrix 계산

In [13]:
from tqdm.notebook import tqdm

# score 계산 함수 정의
def computeScore(pairs, matrix, countsOfD_i) -> np.array:
    for D_i, D_j in tqdm(pairs): # pairs(2차원 배열)에서 한 배열의 요소: [D_i(이전진단코드번호), D_j(이후진단코드번호)]
        if matrix[D_i, D_j] != 0: # 이미 계산 된 진단 쌍이면 다음 사이클 넘어감
            continue
        countOfD_i = countsOfD_i[D_i]
        countOfD_j = np.sum(np.all(pairs == [D_i, D_j], axis=1))
        matrix[D_i, D_j] = countOfD_j / countOfD_i  # 행렬의 각 i, j 인덱스는 진단번호와 같음
    return matrix.round(2)

matrix = np.zeros((3447, 3447), dtype=np.float64) # 3447 x 3447 0행렬 생성

scoreMatrix = computeScore(pairs_number, matrix, countsOfD_i)
scoreMatrix = pd.DataFrame(scoreMatrix, index=diagnosis, columns=diagnosis)

scoreMatrix

  0%|          | 0/134919 [00:00<?, ?it/s]

,H650,J039,K52,H660,J459,J06,K529,J069,J209,H66,...,M6589,M79138,M532,M5327,P599,P59,Q659,P071,P011,S309
H650,0.48,0.02,0.00,0.03,0.01,0.0,0.00,0.01,0.04,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
J039,0.00,0.31,0.00,0.00,0.01,0.0,0.01,0.02,0.06,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
K52,0.00,0.03,0.15,0.03,0.00,0.0,0.00,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
H660,0.04,0.04,0.00,0.40,0.02,0.0,0.00,0.01,0.07,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
J459,0.00,0.02,0.00,0.01,0.40,0.0,0.00,0.01,0.05,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
P59,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Q659,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
P071,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
P011,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Score matrix 엑셀 파일 저장

In [14]:
# scoreMatrix.to_excel('Output/Score_Matrix_All.xlsx')

### 진단쌍 정보 테이블 생성
* ICD-10 질병분류표 불러오기

In [15]:
df_KCD = pd.read_excel('ICD-10.xlsx')
df_KCD

,Diagnosis(ICD-10),Diagnosis(full term)
0,A00,Cholera
1,A000,"Cholera due to Vibrio cholerae 01, biovar chol..."
2,A001,"Cholera due to Vibrio cholerae 01, biovar eltor"
3,A009,"Cholera, unspecified"
4,A01,Typhoid and paratyphoid fevers
...,...,...
41003,M9989/6,"Myelodysplastic syndrome, NOS, malignant, meta..."
41004,M9991/3,"Refractory neutropenia, malignant, primary site"
41005,M9991/6,"Refractory neutropenia, malignant, metastatic ..."
41006,M9992/3,"Refractory thrombocytopenia, malignant, primar..."


CountMatrix 계산

In [16]:
def computeCount(pairs, matrix) -> np.array:
    for D_i, D_j in tqdm(pairs): # pairs(2차원 배열)에서 한 배열의 요소: [D_i(이전진단코드번호), D_j(이후진단코드번호)]
        if matrix[D_i, D_j] != 0: # 이미 계산 된 진단 쌍이면 다음 사이클 넘어감
            continue
        countOfD_j = np.sum(np.all(pairs == [D_i, D_j], axis=1))
        # 행렬의 각 i, j 인덱스는 진단번호와 같음
        matrix[D_i, D_j] = countOfD_j
    return matrix

matrix = np.zeros((3447, 3447), dtype=np.uint16) # 3447 x 3447 0행렬 생성

countMatrix = computeCount(pairs_number, matrix)
countMatrix = pd.DataFrame(countMatrix, index=diagnosis, columns=diagnosis)

countMatrix

  0%|          | 0/134919 [00:00<?, ?it/s]

,H650,J039,K52,H660,J459,J06,K529,J069,J209,H66,...,M6589,M79138,M532,M5327,P599,P59,Q659,P071,P011,S309
H650,149,5,0,8,3,0,0,3,11,1,...,0,0,0,0,0,0,0,0,0,0
J039,6,808,1,7,17,1,22,47,164,3,...,0,0,0,0,0,0,0,0,0,0
K52,0,1,6,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
H660,12,12,0,137,6,0,1,2,25,1,...,0,0,0,0,0,0,0,0,0,0
J459,4,16,0,7,426,1,4,11,53,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
P59,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Q659,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
P071,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
P011,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


정보 테이블 생성
* 이전 진단코드 || 이전 진단명 || 이후 진단코드 || 이후 진단명 || Count || Count ratio

In [17]:
pairs_name = np.array([[D[1], D[3]] for D in np_patients_concat]) # 진단(진단코드명) 쌍 생성

# 진단쌍 데이터프레임 생성
df_pair = pd.DataFrame(pairs_name, columns=[['Prior diagnosis(ICD-10 code)', 'Later diagnosis(ICD-10 code)']])
df_pair = df_pair.drop_duplicates() # 중복되는 진단 쌍 제거
df_pair.reset_index(inplace=True)
     
# ICD 코드 진단명 매핑
df_pair_prev = df_pair[['Prior diagnosis(ICD-10 code)']]
df_pair_next = df_pair[['Later diagnosis(ICD-10 code)']]

df_pair_prev.columns = ['Diagnosis(ICD-10)']
df_pair_next.columns = ['Diagnosis(ICD-10)']

df_pair_prev_merge = pd.merge(df_pair_prev, df_KCD, how='left', on='Diagnosis(ICD-10)')
df_pair_next_merge = pd.merge(df_pair_next, df_KCD, how='left', on='Diagnosis(ICD-10)')

df_pairInfo = pd.concat([df_pair_prev_merge, df_pair_next_merge], axis=1)

df_pairInfo.columns = ['Prior diagnosis(ICD-10)', 'Prior diagnosis(full term)', 
                       'Later diagnosis(ICD-10)', 'Later diagnosis(full term)']

# Count, Count ratio 값 삽입
df_pairInfo['Count'] = 0
df_pairInfo['Count ratio'] = 0.0
for i in range(len(df_pairInfo)):
    df_pairInfo.iloc[i, 4] = countMatrix.iloc[dig_dict[df_pairInfo.iloc[i, 0]], dig_dict[df_pairInfo.iloc[i, 2]]] # matrix(인덱스(=진단번호), 인덱스)
    df_pairInfo.iloc[i, 5] = scoreMatrix.iloc[dig_dict[df_pairInfo.iloc[i, 0]], dig_dict[df_pairInfo.iloc[i, 2]]]

df_pairInfo

,Prior diagnosis(ICD-10),Prior diagnosis(full term),Later diagnosis(ICD-10),Later diagnosis(full term),Count,Count ratio
0,H650,Acute serous otitis media,J039,"Acute tonsillitis, unspecified",5,0.02
1,J039,"Acute tonsillitis, unspecified",K52,Other noninfective gastroenteritis and colitis,1,0.00
2,K52,Other noninfective gastroenteritis and colitis,H660,Acute suppurative otitis media,1,0.03
3,H660,Acute suppurative otitis media,J459,"Asthma, unspecified",6,0.02
4,J459,"Asthma, unspecified",J06,Acute upper respiratory infections of multiple...,1,0.00
...,...,...,...,...,...,...
50726,K59,Other functional intestinal disorders,J20,Acute bronchitis,1,0.04
50727,H66,Suppurative and unspecified otitis media,L210,Seborrhoea capitis,1,0.04
50728,J209,"Acute bronchitis, unspecified",S309,"Superficial injury of abdomen, lower back and ...",1,0.00
50729,S309,"Superficial injury of abdomen, lower back and ...",J209,"Acute bronchitis, unspecified",1,1.00


In [18]:
# df_pairInfo.to_excel('Output/diagnosis pairs information_All.xlsx', index=False)